
L'obiettivo del progetto è apprendere mediante una rete neurale la trasformazione da punti del piano espressi in coordinate polari ad una rappresentazione basata su di una griglia discreta di dimensione 10x10, dove la cella della griglia ha valore 1 se contiene il punto, e 0 altrimenti.

Il dataset supervisionato è fornito in questo notebook nella forma di una generatore. Il generatore deve essere considerato come una "scatola nera" il cui comportamento deve essere appreso. 

Dovete progettare una rete neurale in grado di raggiungere una accuratezza del 95%. Questa è una condizione necessaria per superare l'esame, ma l'accuratezza non influisce in altro modo sulla valutazione.  

I modelli che raggiungono l'accuratezza attesa saranno invece valutati in modo inversamente proporzionale al numero dei loro parametri: **più il modello è piccolo, meglio è.**


**Attenzione**: Qualunque soluzione che tragga vantaggio, diretto o indiretto, da meta-conoscenza relativa al generatore sarà automaticamente bocciato.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import activations


Veniamo al generatore. Questo restituisce delle triple della forma
((theta,rho),out) dove (theta,rho) sono le coordinate polari di un punto nel primo quadrante del piano, e out è una mappa 10x10 con "1" in correspondenza alla cella che contiene il punto, e "0" altrimenti.

Settando  flat=True, la mappa 10x10 viene appiattita ad un vettore di dimensione 100. Potete utilizzare questa variante, se preferite. Nessuna altra modifica del generatore è ammessa. 

In [ ]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Creiamo una istanza del generatore con una griglia di dimensione 3x4

In [ ]:
g1,g2 = 10,10
gen = polar_generator(1,grid=(g1,g2),noise=0.002,flat=False)

... e osserviamo qualche esempio

In [ ]:
(theta,rho),maps = next(gen)
for i,map in enumerate(maps):
  #let us compute the cartesian coordinates
  x = np.cos(theta[i])*rho[i]
  y = np.sin(theta[i])*rho[i]
  print("x coordinate (row): {}".format(int(x*g1)))
  print("y coordinate (col): {}".format(int(y*g2)))
  print("map:")
  print(np.reshape(map,(g1,g2)))

x coordinate (row): 1
y coordinate (col): 1
map:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Utile esercizio: aggiungete rumore al generatore e verificate l'effetto sulla
"ground truth".

# Cosa consegnare

Ai fini del progetto dovete lavorare con la **griglia di default di dimensione 10x10, e con il rumore di default .002**

il generatore deve essere trattato come una scatola nera: non modificatelo e non sfruttate la sua semantica, che si suppone ignota. Potete lavorare in modlaità "flat", se preferite.

Dovete:

1.   definire una funzione per il calcolo della accuratezza (potete prendere ispirazione dal cocice della cella precedente) 
2.   definire una rete neurale che prende in input theta e rho e restituisce out
3.  misurare l'accuratezza della rete, che deve essere maggiore o uguale del 95%; l'accuratezza deve essere misurata su almeno 20000 dati
4. perfezionare il modello cercando di diminuire il più possibile il numero dei parametri mantenendo una accuratezza superiore al 95%. Solo la vostra rete migliore deve essere consegnata.

Dovete consegnare un UNICO notebook eseguibile su colab, che contenga il codice della rete, il suo sommario con il numero dei parametri, la storia di training, il codice per il calcolo della accuratezza e la sua valutazione sulla vostra rete.

**N.B.** L'accuratezza deve essere superiore o uguale a 95%, ma non influisce in altro modo sulla valutazione. Il vostro punteggio dipenderà unicamente dal numero dei parametri: più è piccolo e più la vostra vaalutazione sarà elevata.  

#Buon lavoro!





In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import activations
import keras
from keras import layers
from keras import activations
from sklearn.model_selection import train_test_split

def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

In [ ]:
def my_accuracy(true_maps: tf.Tensor, my_maps: tf.Tensor) -> float:
  equals = tf.equal(tf.argmax(true_maps, axis=1), tf.argmax(my_maps, axis=1))
  return tf.cast(tf.math.count_nonzero(equals), tf.float64) / tf.cast(len(true_maps), tf.float64)

g1,g2 = 10,10
n_train = 4000000
n_test = 20000
batch_size = 1024
epochs = 200

In [ ]:
!unzip crestanello_polaretto.zip

Archive:  crestanello_polaretto.zip
  inflating: crestanello_polaretto/fingerprint.pb  
  inflating: crestanello_polaretto/saved_model.pb  
  inflating: crestanello_polaretto/keras_metadata.pb  
   creating: crestanello_polaretto/assets/
  inflating: crestanello_polaretto/variables/variables.index  
  inflating: crestanello_polaretto/variables/variables.data-00000-of-00001  


# Setup

In [ ]:
gen = polar_generator(n_train+n_test,grid=(g1,g2),noise=0.002,flat=True)
(theta,rho),y = next(gen)

x=np.array([i for i in zip(theta,rho)])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=n_test/(n_train+n_test), shuffle=True, random_state=1)


network = keras.Sequential([
  layers.Flatten(),
  layers.Dense(4, activation=keras.activations.swish),
  layers.Dense(8, activation=keras.activations.relu),
  layers.Dense(4, activation=keras.activations.relu),
  layers.Dense(6, activation=keras.activations.swish),
  layers.Dense(100, activation=activations.softmax)
])

network.build((None, 2))
network.summary()

network.compile(
  optimizer=keras.optimizers.Adam(learning_rate=1e-3),
  loss=keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy', my_accuracy]
)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 2)                 0         
                                                                 
 dense_40 (Dense)            (None, 4)                 12        
                                                                 
 dense_41 (Dense)            (None, 8)                 40        
                                                                 
 dense_42 (Dense)            (None, 4)                 36        
                                                                 
 dense_43 (Dense)            (None, 6)                 30        
                                                                 
 dense_44 (Dense)            (None, 100)               700       
                                                                 
Total params: 818
Trainable params: 818
Non-trainable 

# Training della rete

In [ ]:
history = network.fit(
  x=x_train,
  y=y_train,
  epochs=epochs,
  batch_size=batch_size, 
  validation_data=(x_test, y_test),
  callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)

score, _, acc  = network.evaluate(x_test, y_test,batch_size=batch_size)

print('▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼')
print('Test score:', score)
print('Accuracy: {:.1f}%'.format(acc*100))
print('▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼')

network.summary()


Epoch 1/200
3907/3907 [==============================] - 6s 1ms/step - loss: 2.5507 - accuracy: 0.1888 - my_accuracy: 0.1888 - val_loss: 1.7750 - val_accuracy: 0.3701 - val_my_accuracy: 0.3699
Epoch 2/200
3907/3907 [==============================] - 6s 1ms/step - loss: 0.7850 - accuracy: 0.7314 - my_accuracy: 0.7315 - val_loss: 0.3104 - val_accuracy: 0.9014 - val_my_accuracy: 0.9011
Epoch 3/200
3907/3907 [==============================] - 6s 1ms/step - loss: 0.2364 - accuracy: 0.9292 - my_accuracy: 0.9292 - val_loss: 0.1917 - val_accuracy: 0.9411 - val_my_accuracy: 0.9412
Epoch 4/200
3907/3907 [==============================] - 6s 1ms/step - loss: 0.1710 - accuracy: 0.9485 - my_accuracy: 0.9485 - val_loss: 0.1637 - val_accuracy: 0.9468 - val_my_accuracy: 0.9465
Epoch 5/200
3907/3907 [==============================] - 6s 1ms/step - loss: 0.1430 - accuracy: 0.9561 - my_accuracy: 0.9561 - val_loss: 0.1362 - val_accuracy: 0.9536 - val_my_accuracy: 0.9532
Epoch 6/200
3907/3907 [============

In [ ]:
network.save("crestanello_polaretto")

# Testing

In [ ]:
network = keras.models.load_model("crestanello_polaretto",custom_objects={"my_accuracy":my_accuracy})

network.compile(
  optimizer=keras.optimizers.Adam(learning_rate=1e-3),
  loss=keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy', my_accuracy]
)

In [ ]:
network.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 2)                 0         
                                                                 
 dense_15 (Dense)            (None, 4)                 12        
                                                                 
 dense_16 (Dense)            (None, 8)                 40        
                                                                 
 dense_17 (Dense)            (None, 4)                 36        
                                                                 
 dense_18 (Dense)            (None, 6)                 30        
                                                                 
 dense_19 (Dense)            (None, 100)               700       
                                                                 
Total params: 818
Trainable params: 818
Non-trainable 

In [ ]:
gen = polar_generator(20000,grid=(g1,g2),noise=0.002,flat=True)

iters = 1000
accs = 0.0
lower = 0
batch = 8192
for x in range(iters):
  (theta,rho),y = next(gen)
  x=np.array([i for i in zip(theta,rho)])

  score, _, acc = network.evaluate(x, y, batch_size=batch)
  accs += acc
  if acc < 0.95:
    lower += 1

print('▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼')
print('Accuracy: {:.1f}%'.format(accs/iters*100))
print('Lower than 95%: {}/{} ({:.1f}%)'.format(lower, iters, (lower/iters*100)))
print('▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼')

3/3 [==============================] - 0s 12ms/step - loss: 0.1466 - accuracy: 0.9416 - my_accuracy: 0.9423
▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼
Accuracy: 96.0%
Lower than 95%: 146/1000 (14.6%)
▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼△▼


In [ ]:
network.save("crestanello_polaretto")

In [ ]:
!tar czfv crestanello_model.tar.gz crestanello_polaretto/

crestanello_polaretto/
crestanello_polaretto/assets/
crestanello_polaretto/fingerprint.pb
crestanello_polaretto/variables/
crestanello_polaretto/variables/variables.index
crestanello_polaretto/variables/variables.data-00000-of-00001
crestanello_polaretto/saved_model.pb
crestanello_polaretto/keras_metadata.pb
